In [4]:
import os
from tqdm import tqdm
from src.data.data_for_finetuning import save_correct_answer, find_matching_square, find_rectangles_without_plus
from src.data.preprocess_labels import save_bottom_half

In [5]:
import json 

def find_rectangles_without_plus(path_to_json: str) -> list:
    """
    Находит координаты прямоугольников, не содержащих знак плюса, в файле JSON с аннотациями.

    Аргументы:
    - path_to_json: Путь к файлу JSON с аннотациями.

    Возвращает:
    - Список координат для каждого прямоугольника, не содержащего знак плюса.
      Каждый набор координат представлен в формате списка из двух точек.
    """
    
    with open(path_to_json) as json_file:
        data = json.load(json_file)
    
    # Находим координаты всех знаков плюса
    plus_signs = [shape['points'][0] for shape in data['shapes'] if shape['label'] == '+']

    # Функция для проверки, находится ли точка внутри прямоугольника
    def is_point_in_rect(point, rect):
        px, py = point
        (rx1, ry1), (rx2, ry2) = rect
        return rx1 <= px <= rx2 and ry1 <= py <= ry2

    # Список для хранения всех прямоугольников
    all_rectangles = []

    # Список для хранения прямоугольников, содержащих плюс
    rectangles_with_plus = []

    # Сначала получаем все прямоугольники
    for shape in data['shapes']:
        if (shape['shape_type'] == 'rectangle') and not (shape['label'] in ["1", "2", "3", "4", "5"]) :
            all_rectangles.append(shape['points'])
            #print(shape)

    # Определяем прямоугольники с знаком плюс
    for rect in all_rectangles:
        for plus in plus_signs:
            if is_point_in_rect(plus, rect):
                rectangles_with_plus.append(rect)
                #print("rejected: ", shape)
                break  # Нет необходимости проверять другие знаки плюс, если один найден

    # Теперь получаем список прямоугольников без знака плюс путем удаления rectangles_with_plus из all_rectangles
    rectangles_without_plus = [rect for rect in all_rectangles if rect not in rectangles_with_plus]

    return rectangles_without_plus

In [12]:
path_to_json = 'data/raw/186586.json'
len(find_rectangles_without_plus(path_to_json))

{'label': 'icon', 'points': [[155.1558752997602, 39.32853717026379], [108.39328537170265, 0.0]], 'group_id': None, 'description': '', 'shape_type': 'rectangle', 'flags': {}}
{'label': 'icon', 'points': [[102.6378896882494, 26.618705035971225], [151.3189448441247, 72.18225419664269]], 'group_id': None, 'description': '', 'shape_type': 'rectangle', 'flags': {}}
{'label': 'icon', 'points': [[75.77937649880096, 47.48201438848921], [126.13908872901679, 89.68824940047962]], 'group_id': None, 'description': '', 'shape_type': 'rectangle', 'flags': {}}
{'label': 'icon', 'points': [[81.5347721822542, 80.3357314148681], [129.97601918465227, 129.97601918465227]], 'group_id': None, 'description': '', 'shape_type': 'rectangle', 'flags': {}}
{'label': 'icon', 'points': [[50.35971223021583, 98.8009592326139], [100.71942446043165, 148.4412470023981]], 'group_id': None, 'description': '', 'shape_type': 'rectangle', 'flags': {}}
{'label': 'icon', 'points': [[96.64268585131894, 375.53956834532374], [40.52

5